## Ames Kaggle Dataset

Initial inspection of the data and EDA to choose the best variables for the analysis

In [43]:
#Libraries we will be using

import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
plotly.tools.set_credentials_file(username='natxomoreno', api_key='FUB7h2j9TcTcXLl7ak7D')
plotly.tools.set_config_file(world_readable=True,
                             sharing='public')


In [22]:
#import and read the train data

train = pd.read_csv('train_enriched.csv')

shape = train.shape
print(shape)

print(train.describe())

train.dtypes



(1460, 93)
                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726   
std       1.112799    30.202904     20.645407   181.066207   456.098091   
min 

Id                                         int64
MSSubClass                                 int64
MSZoning                                  object
LotFrontage                              float64
LotArea                                    int64
Street                                    object
Alley                                     object
LotShape                                  object
LandContour                               object
Utilities                                 object
LotConfig                                 object
LandSlope                                 object
Neighborhood                              object
Condition1                                object
Condition2                                object
BldgType                                  object
HouseStyle                                object
OverallQual                                int64
OverallCond                                int64
YearBuilt                                  int64
YearRemodAdd        

### Sale Price Data by Neighbourhood

In [84]:
#Make sure there is no missing values in the SalePrice Column

train['SalePrice'].count()

#seems there is no missing data in SalePrice

1460

In [115]:
#First histogram to see the distribution of the data
import plotly.graph_objs as go

#Normal histogram
x = train['SalePrice']
data = [go.Histogram(x=x)]

py.iplot(data)



In [77]:
#Lets do a boxplot to detect outliers

import plotly.graph_objs as go

layout = go.Layout(
    autosize=False,
    width=500,
    height=500)
data = [
    go.Box(
        y=train['SalePrice'],
        boxpoints='outliers',
        name = 'Sale Price',
        marker=dict(size=3, 
                    color = 'rgb(0,0,0)'),
        jitter=0.3,
        pointpos=0,
        line = dict(width=2,
                    color = 'rgb(227,6,19)')
    )
]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)


In [79]:
#Lets get some stats about the skewness for the whole sample

train['SalePrice'].skew()

1.8828757597682129

In [113]:
#Lets see the skew for each of the neighborhoods

print(train['SalePrice'].groupby(train['Neighborhood']).skew())

train['SalePrice'].groupby(train['Neighborhood']).count()

Neighborhood
Blmngtn    1.038435
Blueste         NaN
BrDale    -0.109211
BrkSide    0.470675
ClearCr    0.392387
CollgCr    0.928094
Crawfor    0.782491
Edwards    1.612723
Gilbert    2.611472
IDOTRR    -0.235471
MeadowV    1.093526
Mitchel    0.942499
NAmes      1.830231
NPkVill   -0.843657
NWAmes     0.586247
NoRidge    2.357491
NridgHt    0.736906
OldTown    3.517007
SWISU     -0.323101
Sawyer    -0.418470
SawyerW    0.400760
Somerst    0.867115
StoneBr    0.778667
Timber     0.518700
Veenker    0.862404
Name: SalePrice, dtype: float64


Neighborhood
Blmngtn     17
Blueste      2
BrDale      16
BrkSide     58
ClearCr     28
CollgCr    150
Crawfor     51
Edwards    100
Gilbert     79
IDOTRR      37
MeadowV     17
Mitchel     49
NAmes      225
NPkVill      9
NWAmes      73
NoRidge     41
NridgHt     77
OldTown    113
SWISU       25
Sawyer      74
SawyerW     59
Somerst     86
StoneBr     25
Timber      38
Veenker     11
Name: SalePrice, dtype: int64

There is a huge disparity between different neighborhoods.

Lets delete some of the outliers we saw initially and see how this values change

In [136]:
#Remove Outliers with more than 3 std

sale_price = train[['SalePrice']]

sale_price = sale_price.loc[sale_price.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

#how many outliers have we deleted?
count_outliers = (train['SalePrice'].count() - sale_price.count())
ratio_outliers = (count_outliers / train['SalePrice'].count())*100
print(count_outliers)
print(ratio_outliers)

#Max price
max_price = sale_price['SalePrice'].max()
print(max_price)

#skewness is greatly reduced
print(sale_price.skew())
sale_price.kurtosis()

SalePrice    22
dtype: int64
SalePrice    1.506849
dtype: float64
415298
SalePrice    1.001684
dtype: float64


SalePrice    0.949597
dtype: float64

In [119]:
#lets visualize this new distribution

x = sale_price['SalePrice']
data = [go.Histogram(x=x)]

py.iplot(data)



In [128]:
#Since data is right skewed we will create a new duplicated Sale_Price column and apply a log transformation

train['logSalePrice'] = np.log(train['SalePrice'])

pd.set_option('display.max_columns', None)
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Average Household Income,Median Age,Median Household Income,Per Capita Income,Food Stores (SIC54),Eating & Drinking (SIC58),Hotel/Lodging (SIC70),Dominant Tapestry LifeMode Group Code,Dominant Tapestry LifeMode Group Name,Total Population,Buffer Distance,Buffer Units,logSalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,59493,24.2,52597,25185,4,7,1,14,Scholars and Patriots,5547,1,Kilometers,12.247694
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,77663,28.3,53564,34486,0,4,0,2,Upscale Avenues,3923,1,Kilometers,12.109011
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,59493,24.2,52597,25185,4,7,1,14,Scholars and Patriots,5547,1,Kilometers,12.317167
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,44200,21.9,25259,13158,3,16,1,14,Scholars and Patriots,8383,1,Kilometers,11.849398
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,111096,37.0,93059,46810,0,4,0,2,Upscale Avenues,3062,1,Kilometers,12.429216


In [134]:
#Let's check teh results

train['logSalePrice'].skew()

#Histogram

x = train['logSalePrice']
data = [go.Histogram(x=x)]

py.iplot(data)


